This script extracts Gray-Level Co-occurrence Matrix (GLCM) texture features from paired diffraction images (P and S polarization) of cells.

In [ ]:
# -*- coding: utf-8 -*-
"""
GLCM Feature Extraction + Multi-Classifier Evaluation (SVM OvR & Random Forest)
Description: This script extracts GLCM features from paired P/S diffraction images, 
saves the features to CSV, and evaluates multiple classifiers (SVM with linear, poly, sigmoid, rbf kernels, and Random Forest)
using train/validation/test split, cross-validation, confusion matrices, and pairwise accuracy matrices.
"""

import os, glob, time, random
import numpy as np
import pandas as pd
import cv2
import tifffile
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from skimage.feature import greycomatrix, greycoprops
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# =========================
# Configuration
# =========================
data_dirs = {
    "Healthy":   r"E:\0. Research\1. Paper_data\2. Hacat cisplatin ROS\数据集\原始数据\downsample-healthy",
    "Cisplatin": r"E:\0. Research\1. Paper_data\2. Hacat cisplatin ROS\数据集\原始数据\Cisplatin",
    "H2O2":      r"E:\0. Research\1. Paper_data\2. Hacat cisplatin ROS\数据集\原始数据\H2O2"
}
resize_size = 128
num_threads = 8  # Adjust according to CPU cores

angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
distances = [1, 2, 4, 8, 16, 32]
properties = ['ASM', 'contrast', 'correlation', 'homogeneity', 'dissimilarity', 'energy']

# =========================
# Helper functions
# =========================
def get_image_pairs(folder):
    """Get all P/S image pairs in a folder"""
    p_files = sorted(glob.glob(os.path.join(folder, "*-P*.tif*")))
    pairs = []
    for p_file in p_files:
        s_file = p_file.replace("-P", "-S")
        if os.path.exists(s_file):
            pairs.append((p_file, s_file))
    return pairs

def extract_glcm_features(img):
    """Extract GLCM features from a single image (normalized to 0~1)"""
    img_uint8 = (img * 255).astype(np.uint8)
    glcm = greycomatrix(img_uint8, distances=distances, angles=angles, symmetric=True, normed=True)
    feats = []
    for prop in properties:
        feats.extend(greycoprops(glcm, prop).flatten())
    return feats  # 144 features

def process_pair(pair):
    """Extract combined GLCM features from a P/S image pair"""
    p_file, s_file, label = pair
    try:
        p_img = cv2.resize(tifffile.imread(p_file).astype(np.float32)/65535., (resize_size, resize_size))
        s_img = cv2.resize(tifffile.imread(s_file).astype(np.float32)/65535., (resize_size, resize_size))
        p_feats = extract_glcm_features(p_img)
        s_feats = extract_glcm_features(s_img)
        return [label] + p_feats + s_feats
    except Exception as e:
        print(f"Error processing {p_file} / {s_file}: {e}")
        return None

# =========================
# Build list of all image pairs
# =========================
all_pairs = []
for label, folder in data_dirs.items():
    pairs = get_image_pairs(folder)
    print(f"{label} folder contains {len(pairs)} image pairs")
    all_pairs.extend([(p, s, label) for p, s in pairs])

# =========================
# Parallel GLCM feature extraction
# =========================
results = []
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    futures = [executor.submit(process_pair, pair) for pair in all_pairs]
    for f in tqdm(as_completed(futures), total=len(futures), desc="Processing all pairs"):
        res = f.result()
        if res is not None:
            results.append(res)

# =========================
# Save features to CSV
# =========================
columns = ['Label'] + [f'feat_{i}' for i in range(288)]
df = pd.DataFrame(results, columns=columns)
df.to_csv('glcm_features_multithread.csv', index=False)
print("GLCM features saved to glcm_features_multithread.csv")

# =========================
# Load GLCM features
# =========================
df = pd.read_csv('glcm_features_multithread.csv')
X = df.drop("Label", axis=1).values
y = df["Label"].values
class_names = np.unique(y)

# =========================
# Train / Validation / Test Split (8:1:1)
# =========================
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.1111, stratify=y_trainval, random_state=42
)  # ~8:1:1
print(f"Train: {len(y_train)}, Validation: {len(y_val)}, Test: {len(y_test)}")

# Standardization
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

# =========================
# Visualization functions
# =========================
def plot_confusion_matrix(y_true, y_pred, classes, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

def plot_pairwise_accuracy(y_true, y_pred, classes, title="Pairwise Accuracy Matrix"):
    n_classes = len(classes)
    pairwise_acc = np.zeros((n_classes, n_classes))
    for i in range(n_classes):
        for j in range(n_classes):
            if i == j:
                pairwise_acc[i, j] = 1.0
            elif i < j:
                mask = np.isin(y_true, [classes[i], classes[j]])
                if np.sum(mask) > 0:
                    acc = np.mean(y_true[mask] == y_pred[mask])
                    pairwise_acc[i, j] = pairwise_acc[j, i] = acc
    plt.figure(figsize=(6, 5))
    sns.heatmap(pairwise_acc, annot=True, fmt=".4f", xticklabels=classes, yticklabels=classes, cmap="Blues")
    plt.title(title)
    plt.show()

# =========================
# Define classifiers
# =========================
models = {
    "SVM-linear-OvR": SVC(kernel="linear", probability=True, random_state=42,
                          decision_function_shape='ovr'),
    "SVM-poly-OvR":   SVC(kernel="poly", degree=3, probability=True, random_state=42,
                          decision_function_shape='ovr'),
    "SVM-sigmoid-OvR":SVC(kernel="sigmoid", probability=True, random_state=42,
                          decision_function_shape='ovr'),
    "SVM-rbf-OvR":    SVC(kernel="rbf", probability=True, random_state=42,
                          decision_function_shape='ovr'),
    "RandomForest":   RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
}

# =========================
# Training and Evaluation
# =========================
skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"\n===== {name} =====")
    
    # Cross-validation on train+validation
    y_pred_cv = cross_val_predict(model, np.vstack([X_train, X_val]), np.hstack([y_train, y_val]), cv=skf)
    print("Cross-validation classification report:")
    print(classification_report(np.hstack([y_train, y_val]), y_pred_cv, target_names=class_names, digits=4))
    
    # Train on train+validation, then evaluate on test
    model.fit(np.vstack([X_train, X_val]), np.hstack([y_train, y_val]))
    y_pred_test = model.predict(X_test)
    print("Test classification report:")
    print(classification_report(y_test, y_pred_test, target_names=class_names, digits=4))
    
    # Confusion matrix
    plot_confusion_matrix(y_test, y_pred_test, class_names, title=f"{name} - Confusion Matrix (Test)")
    
    # Pairwise accuracy
    plot_pairwise_accuracy(y_test, y_pred_test, class_names, title=f"{name} - Pairwise Accuracy Matrix (Test)")


"""
SVM-Linear C Parameter Grid Search on GLCM Features
Description: This script performs a grid search to find the optimal C parameter for a linear SVM
using GLCM features, evaluates on a test set, and plots confusion and pairwise accuracy matrices.
"""

In [ ]:


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC

# =========================
# Load GLCM features
# =========================
df = pd.read_csv("glcm_merged_normalized_multithread.csv")
X = df.drop("Label", axis=1).values
y = df["Label"].values
class_names = np.unique(y)

# =========================
# Train / Validation / Test Split (8:1:1)
# =========================
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.1111, stratify=y_trainval, random_state=42
)  # ~8:1:1

print(f"Train: {len(y_train)}, Validation: {len(y_val)}, Test: {len(y_test)}")

# =========================
# Standardization
# =========================
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

X_trainval_scaled = np.vstack([X_train, X_val])
y_trainval = np.hstack([y_train, y_val])

# =========================
# Confusion Matrix Plotting
# =========================
def plot_confusion_matrix(y_true, y_pred, classes, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred, labels=classes)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title(title)
    plt.ylabel("True Label")
    plt.xlabel("Predicted Label")
    plt.show()

# =========================
# Pairwise Accuracy Plotting
# =========================
def plot_pairwise_accuracy(y_true, y_pred, classes, title="Pairwise Accuracy Matrix"):
    n_classes = len(classes)
    pairwise_acc = np.zeros((n_classes, n_classes))

    for i in range(n_classes):
        for j in range(n_classes):
            if i == j:
                pairwise_acc[i, j] = 1.0
            elif i < j:
                mask = np.isin(y_true, [classes[i], classes[j]])
                if np.sum(mask) > 0:
                    acc = np.mean(y_true[mask] == y_pred[mask])
                    pairwise_acc[i, j] = pairwise_acc[j, i] = acc

    plt.figure(figsize=(6, 5))
    sns.heatmap(pairwise_acc, annot=True, fmt=".4f", xticklabels=classes, yticklabels=classes, cmap="Blues")
    plt.title(title)
    plt.show()

# =========================
# Grid Search for Optimal C (Linear SVM)
# =========================
C_values = list(range(1, 252, 10))  # C = 1, 11, ..., 241
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

mean_accuracies = []
for C in C_values:
    model = SVC(kernel="linear", C=C, random_state=42)
    scores = cross_val_score(model, X_trainval_scaled, y_trainval, cv=cv, scoring="accuracy")
    mean_accuracies.append(scores.mean())

# Find the best C
best_idx = np.argmax(mean_accuracies)
best_C = C_values[best_idx]
best_acc = mean_accuracies[best_idx]

print(f"Optimal C = {best_C}, Cross-Validation Accuracy = {best_acc:.4f}")

# =========================
# Plot Accuracy vs C
# =========================
plt.figure(figsize=(8, 5))
plt.plot(C_values, mean_accuracies, marker="o")
plt.axvline(best_C, color="red", linestyle="--", label=f"Best C={best_C}")
plt.xlabel("C value")
plt.ylabel("Cross-Validation Accuracy")
plt.title("SVM (linear) - Accuracy vs C")
plt.legend()
plt.grid(True)
plt.show()

# =========================
# Train on Full Train+Val with Best C and Evaluate on Test
# =========================
final_model = SVC(kernel="linear", C=best_C, probability=True, random_state=42)
final_model.fit(X_trainval_scaled, y_trainval)
y_pred_test = final_model.predict(X_test)

print("\nTest Classification Report:")
print(classification_report(y_test, y_pred_test, target_names=class_names, digits=4))

# Confusion Matrix
plot_confusion_matrix(y_test, y_pred_test, class_names, title=f"SVM (linear, C={best_C}) - Confusion Matrix (Test)")

# Pairwise Accuracy Matrix
plot_pairwise_accuracy(y_test, y_pred_test, class_names, title=f"SVM (linear, C={best_C}) - Pairwise Accuracy Matrix (Test)")


In [ ]:
Benchmarking GLCM + SVM Inference Time on Test Set
"""

In [ ]:
# -*- coding: utf-8 -*-


import pandas as pd
import numpy as np
import cv2
import tifffile
import time
from tqdm import tqdm
from skimage.feature import greycomatrix, greycoprops
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# =========================
# Configuration
# =========================
CSV_PATH = r"glcm_merged_normalized_multithread.csv"
resize_size = 128

angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
distances = [1, 2, 4, 8, 16, 32]
properties = ['ASM', 'contrast', 'correlation', 'homogeneity', 'dissimilarity', 'energy']

# =========================
# Load CSV with features and image paths
# =========================
df = pd.read_csv(CSV_PATH)
labels = df["Label"].values
p_paths = df["P_Path"].values
s_paths = df["S_Path"].values
features = df.drop(["Label", "P_Path", "S_Path"], axis=1).values

# =========================
# Split Train/Validation/Test sets (test only used here)
# =========================
X_trainval, X_test, y_trainval, y_test, p_trainval, p_test, s_trainval, s_test = train_test_split(
    features, labels, p_paths, s_paths,
    test_size=0.1, stratify=labels, random_state=42
)

# =========================
# Standardize + Train SVM with best C
# =========================
scaler = StandardScaler()
X_trainval_scaled = scaler.fit_transform(X_trainval)
X_test_scaled = scaler.transform(X_test)

best_C = 21  
svm_model = SVC(kernel="linear", C=best_C, probability=False)
svm_model.fit(X_trainval_scaled, y_trainval)

# =========================
# Define GLCM extraction function
# =========================
def extract_glcm_features(img):
    """Extract GLCM features from a single image (normalized to 0-1)"""
    img_uint8 = (img * 255).astype(np.uint8)
    glcm = greycomatrix(img_uint8, distances=distances, angles=angles, symmetric=True, normed=True)
    feats = []
    for prop in properties:
        feats.extend(greycoprops(glcm, prop).flatten())
    return feats

# =========================
# Benchmarking inference on test set
# =========================
glcm_times = []
svm_times = []
full_times = []

for p, s, label in tqdm(zip(p_test, s_test, y_test), total=len(y_test), desc="Benchmarking Test Set"):

    # Read and resize images
    img_p = cv2.resize(tifffile.imread(p).astype(np.float32)/65535., (resize_size, resize_size))
    img_s = cv2.resize(tifffile.imread(s).astype(np.float32)/65535., (resize_size, resize_size))

    # Time GLCM extraction
    t1 = time.perf_counter()
    p_feats = extract_glcm_features(img_p)
    s_feats = extract_glcm_features(img_s)
    feats = np.array(p_feats + s_feats).reshape(1, -1)
    t2 = time.perf_counter()
    glcm_times.append((t2 - t1) * 1000)  # ms

    # Standardize features
    feats_scaled = scaler.transform(feats)

    # Time SVM prediction
    t3 = time.perf_counter()
    _ = svm_model.predict(feats_scaled)
    t4 = time.perf_counter()
    svm_times.append((t4 - t3) * 1000)  # ms

    # Full pipeline time
    full_times.append((t4 - t1) * 1000)  # ms

# =========================
# Output benchmarking results
# =========================
print("\n========== Benchmark Results (Test Set) ==========")
print(f"[Full Pipeline] GLCM + SVM")
print(f"  Average: {np.mean(full_times):.3f} ms/sample")
print(f"  Total: {np.sum(full_times)/1000:.2f} sec for {len(full_times)} samples\n")

print(f"[GLCM Only]")
print(f"  Average: {np.mean(glcm_times):.3f} ms/sample\n")

print(f"[SVM Only]")
print(f"  Average: {np.mean(svm_times)*1000:.3f} µs/sample")  # convert to microseconds
